In [98]:
from gensim.models import Word2Vec
import helper
import pandas as pd
import csv 
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics

In [90]:
trainingdata = pd.read_csv("../datasets/train/SemEval2018-T3-train-taskA_emoji.txt", delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)


In [93]:
corpus = helper.allbags(trainingdata['Tweet text'])

In [103]:
model = Word2Vec(corpus, min_count=1)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [124]:
words = model.wv.index2word
X = model.wv[words]

In [130]:
print (model.most_similar(positive=['machine'], negative=[], topn=2))

[('alternative', 0.582001268863678), ('#goodtv', 0.573614776134491)]


/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
NUM_CLUSTERS=2000
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
print (assigned_clusters)
 
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))
